In [1]:
import bs4
import requests
import time
from multiprocessing import Pool

# para Session()
# from requests.adapters import HTTPAdapter
# from requests.packages.urllib3.util.retry import Retry


visitedLinks = set()

def get_links(start):
    links = set()
    links.add(start) 
     
    res = ''
    while res == '':
        try:
            res = requests.get(start)
            break
        except:
            time.sleep(5)
            continue    

# outra opcao   
#     session = requests.Session()
#     retry = Retry(total=1000, connect=1000, read=1000, redirect=1000, backoff_factor=0.5)
#     adapter = HTTPAdapter(max_retries=retry)
#     session.mount(start, adapter)
#     session.mount(start, adapter)
#     res = session.get(start)
    
    soup = bs4.BeautifulSoup(res.text, 'lxml')     
    
    for link in soup.find_all('a', href=True):
        href = link["href"]
    
        # apenas links do site
        if (href.startswith(start) | href.startswith("/")):
        
            # trata urls relativos
            url = start + str(href) 
            
            # trata links para imagens (nao é possivel tratar com content-type)
            if "png" not in url or "jpg" not in url:
                links.add(url)
    
    # remove todos os links que sao iguais a buscas passadas
    links = links.difference(visitedLinks)
    
    # armazena todos os links já consultados sem repeticoes
    visitedLinks.update(links)
    
    return list(links)
    

def crawl_bsf(start): 
    queue = []
    frontier = [start] # start -> frontier 
    count = 0
    
    while(len(frontier) != 0 and count < 1000):  
        seen = frontier.pop(0)        
        queue.append(seen)
        if (len(frontier) <= 1000):
            frontier.extend(get_links(seen)) # adiciona os vizinhos no fim da lista
        count += 1

    print(len(queue)) # quantas pages do site foram crawleadas
    
    return queue

In [ ]:
seeds = ["https://www.minhavida.com.br", "https://medicoresponde.com.br",  "https://emagrecer.eco.br",
             "https://www.tuasaude.com", "https://www.mdsaude.com", "http://portalms.saude.gov.br", 
             "http://www.portalnatural.com", "http://portal.saude.pe.gov.br"] 

# descartados "https://minutosaudavel.com.br" e "http://www.riocomsaude.rj.gov.br"

sites = []

for link in seeds:
    print("Crawling", link, ":")
    sites.append(crawl_bsf(link))  # lista de listas, para armazenar os links por site

Crawling https://www.minhavida.com.br :
1000
Crawling https://medicoresponde.com.br :
1000
Crawling https://emagrecer.eco.br :
